# seedbleniding

exp40,exp49 のseed42,614


In [1]:
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import joblib
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from itertools import combinations

import pickle

In [2]:
train_42 = pd.read_csv("../output/exp51_lgb_lagdiff_c3_statedate/oof_exp51_lgb_0.7974057306116666_5fold_seed42.csv")
test_42 = pd.read_csv("../output/exp51_lgb_lagdiff_c3_statedate/test_exp51_lgb_0.7974057306116666_5fold_seed42.csv")

train_615 = pd.read_csv("../output/exp51_615_lgb_lagdiff_c3_statedate/oof_exp51_lgb_0.7965816551786491_5fold_seed615.csv")
test_615 = pd.read_csv("../output/exp51_615_lgb_lagdiff_c3_statedate/test_exp51_lgb_0.7965816551786491_5fold_seed615.csv")



In [3]:
train_42.head()

,customer_ID,target,tabnet_oot
0,0001812036f1558332e5c0880ecbad70b13a6f28ab04a8...,1,0.776060
1,00031e8be98bc3411f6037cbd4d3eeaf24b3ae221682b7...,0,0.016929
2,0004860c260168fcaad0734a1dfedb7ceb1a83aaac54e2...,1,0.998247
3,0004b8596c4946866d1b27a8b40488ecf49c6eae9c7bf4...,0,0.002437
4,000548e99fa24cef8377e68e602e4bd70d30500a007999...,0,0.001118


In [4]:
train_615.head()

,customer_ID,target,tabnet_oot
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0,0.000358
1,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0,0.000993
2,0000d17a1447b25a01e42e1ac56b091bb7cbb06317be4c...,0,0.001775
3,0001812036f1558332e5c0880ecbad70b13a6f28ab04a8...,1,0.786777
4,00039533fe0b61bcf1ec0d1aefe6acb5469ea0f0d1b0ad...,1,0.777778


In [6]:
oof = train_42.merge(train_615,on = "customer_ID")
sub = test_42.merge(test_615,on = "customer_ID")


In [7]:
sub.head()

,customer_ID,prediction_x,prediction_y
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.025009,0.024471
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.000833,0.000896
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.032796,0.030549
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.226481,0.210089
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.855747,0.858729


In [8]:
oof.head()

,customer_ID,target_x,tabnet_oot_x,target_y,tabnet_oot_y
0,0001812036f1558332e5c0880ecbad70b13a6f28ab04a8...,1,0.776060,1,0.786777
1,00031e8be98bc3411f6037cbd4d3eeaf24b3ae221682b7...,0,0.016929,0,0.014767
2,0004860c260168fcaad0734a1dfedb7ceb1a83aaac54e2...,1,0.998247,1,0.998272
3,0004b8596c4946866d1b27a8b40488ecf49c6eae9c7bf4...,0,0.002437,0,0.001818
4,000548e99fa24cef8377e68e602e4bd70d30500a007999...,0,0.001118,0,0.001266


In [11]:
oof["exp51_42+615_oof"] = (oof["tabnet_oot_x"] + oof["tabnet_oot_y"])/2
sub["prediction"] = (sub["prediction_x"] + sub["prediction_y"])/2 

In [15]:
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)


score = amex_metric(oof["target_x"], oof["exp51_42+615_oof"])
print("CV : ",score)

CV :  0.7977581573285044


In [16]:
oof[["customer_ID","exp51_42+615_oof"]].to_csv(f"exp5_oof_lgb_{score}_seedblend.csv",index = False)
sub[["customer_ID","prediction"]].to_csv(f"exp51_sub_lgb_{score}_seedblend.csv",index = False)


In [14]:

amex_metric(oof["target_x"], oof["tabnet_oot_y"])

0.7965816551786491